In [1]:
from collections import defaultdict, Counter


In [2]:
import sys, os
sys.path.insert(0, os.path.abspath('../'))
from utils.osm_helper import *
import geopandas as gp
from osmread import Node, Way, Relation
import datetime
import pandas as pd
OSM_DC_BBOX_PATH = '../data/osm/osm_dc_bbox.osm'
OSM_DC_BBOX = osm_container(OSM_DC_BBOX_PATH)
print OSM_DC_BBOX.data_size()
DC_OSM_IDS_PATH ='../data/osm/osm_ids_within_dc_polygon.txt'
DC_OSM_IDS={}
with open(DC_OSM_IDS_PATH) as f:
    DC_OSM_IDS = eval(f.readlines()[0])
for key, value in DC_OSM_IDS.items():
    print key, len(value)
    
    
def dc_obj(osm_objs, osm_ids, osmtype, osmtype_str):
    return [obj for obj in osm_objs.osm_objs[osmtype] if obj.id in osm_ids[osmtype_str]]

DC_NODES = dc_obj(OSM_DC_BBOX, DC_OSM_IDS,Node, 'Node')
DC_WAYS = dc_obj(OSM_DC_BBOX, DC_OSM_IDS, Way, 'Way')
DC_RLTNS = dc_obj(OSM_DC_BBOX, DC_OSM_IDS, Relation, 'Relation')
len(DC_WAYS), len(DC_NODES), len(DC_RLTNS)

begin reading osm 2016-12-01 09:44:07.560000
finish reading osm 2016-12-01 09:45:22.481000
["len of <class 'osmread.elements.Node'> = 2546260", "len of <class 'osmread.elements.Way'> = 306332", "len of <class 'osmread.elements.Relation'> = 3427"]
Node 1652906
Relation 2556
Way 194767


(194767, 1652906, 2556)

In [3]:
import sys,os
sys.path.insert(0, os.path.abspath('../'))
from utils.osm.osmFilter import *
from IPython.display import display

In [4]:
# have_tag1 = [('building','yes')]
# have_tag2 = [('addr:housenumber','*')]
# have_tag3 = have_tag1+have_tag2
# filter_osm_data_to_df(DC_WAYS,[('dcgis:shape_area','*')])['dcgis:shape_area'].value_counts().shape
# filter_osm_data(DC_WAYS, have_tag1).__len__(), filter_osm_data(DC_WAYS, have_tag2).__len__(), filter_osm_data(DC_WAYS, have_tag3).__len__()
# filter_osm_data(DC_WAYS, have_tag1, have_tag2)[:20]
# filter_osm_data(filter_osm_data(DC_WAYS, have_tag1), have_tag2)[:10]
# filter_osm_data(DC_WAYS, [('name','*')]).__len__()

In [5]:

def get_new_taxonomy ():
    result = []
    xmind_tree = '../data/osm/taxonomy mapping based on xmind.txt'
    with open(xmind_tree) as f:
        category, tag, value = '','',''
        for line in f:
            line_ = line
            line = line.split('\t')
            type_ = len(line)
            txt = line[-1].strip()
            if type_==1:
                category = txt
            elif type_==2:
                tag = txt
            elif type_==3:
                value = txt
                result.append([category, tag, value])
    return pd.DataFrame(result,columns=['category', 'tag', 'value'])

new_taxonomy = get_new_taxonomy()

mapping = pd.Series(new_taxonomy.category.values, index=new_taxonomy.apply(lambda x: '{}={}'.format(x.tag, x.value), axis=1).values).to_dict()

In [6]:
len(mapping)

224

In [7]:
def taxonomy_distribution(osm_data,mapping):
    category_combination = []
    for d in osm_data:
        mapped = set()
        for tag, value in d.tags.items():
            tag_value = '{}={}'.format(tag.encode('utf-8'), value.encode('utf-8'))
            tag_ = tag+'=*'
            if tag_value in mapping:
                mapped.add(mapping[tag_value])
            elif tag_ in mapping:
                mapped.add(mapping[tag_])
        category_combination.append(mapped)

    return category_combination

way_mapped = taxonomy_distribution(DC_WAYS, mapping)
node_mapped = taxonomy_distribution(DC_NODES, mapping)
rltn_mapped = taxonomy_distribution(DC_RLTNS, mapping)
print pd.Series([w for w in way_mapped if w]).apply(str).value_counts().sum(),\
pd.Series([w for w in node_mapped if w]).apply(str).value_counts().sum(),\
pd.Series([w for w in rltn_mapped if w]).apply(str).value_counts().sum()

print pd.concat([
        pd.Series([w for w in way_mapped if w]).apply(str).value_counts(), 
        pd.Series([w for w in node_mapped if w]).apply(str).value_counts(),
        pd.Series([w for w in rltn_mapped if w]).apply(str).value_counts()
    ], axis=1).sum(axis=1).order(ascending=False)

print pd.concat([
        pd.Series([w for w in way_mapped if w]).apply(len).value_counts(), 
        pd.Series([w for w in node_mapped if w]).apply(len).value_counts(),
        pd.Series([w for w in rltn_mapped if w]).apply(len).value_counts()
    ], axis=1).sum(axis=1).order(ascending=False)

20027 14040 1217
set(['cycling facilities'])                                                       12276
set(['transportation'])                                                            7873
set(['outdoors and recreation'])                                                   5593
set(['exclude'])                                                                   2281
set(['retail shop'])                                                               1712
set(['food'])                                                                      1591
set(['professional service'])                                                      1422
set(['schools&university'])                                                         948
set(['residence'])                                                                  580
set(['cycling facilities', 'outdoors and recreation'])                              278
set(['nightlife spot'])                                                             265
set(['transport

In [11]:
a = zip([x.id for x in DC_RLTNS], ['Relation']*len(DC_RLTNS), rltn_mapped)+\
zip([x.id for x in DC_WAYS], ['Way']*len(DC_WAYS), way_mapped)+\
zip([x.id for x in DC_NODES], ['Node']*len(DC_NODES), node_mapped)
print len(a)
a = pd.DataFrame(a,columns=['id','type','category'])
a.to_csv('../data/osm/osm_category_in_dc.txt')

1850229


In [11]:
34521+758*2+15

36052

In [12]:
tags_in_subtree = eval(open('../data/osm/osm_tags_in_subtrees.txt').readlines()[0])

In [13]:

untransformed_tag_value = defaultdict(int)
mapped_subtrees = []
for tags in tags_in_subtree:
    subtree = set()
    for tag, value_list in tags.items():
        for value in list(set(value_list)):
            tag_value = '{}={}'.format(tag.encode('utf-8'), value.encode('utf-8'))
            tag_ = tag+'=*'
            if tag_value in mapping:
                subtree.add(mapping[tag_value])
            elif tag_ in mapping:
                subtree.add(mapping[tag_])
            else:
                untransformed_tag_value[tag_value]+=1
    mapped_subtrees.append(subtree)

In [33]:
with open('../data/osm/osm_tags_mapped_in_subtrees.txt', 'w') as f:
    f.write(str(mapped_subtrees))

In [14]:
for cnt, tags in enumerate(tags_in_subtree):
    for tag, value_list in tags.items():
        if tag=='name' and 'Mary McLeod Bethune Council House' in value_list:
            print cnt
            break

91225
95134


In [15]:
new_mapped = [s for s in mapped_subtrees if s]

In [16]:
un_mapped = [t for cnt, t in enumerate(tags_in_subtree) if not mapped_subtrees[cnt]]

In [6]:
word_in_name = defaultdict(int)
for w in DC_WAYS:
    if 'name' in w.tags:
        for word in w.tags['name'].split(' '):
            word_in_name[word.lower()]+=1
# for key, value in word_in_name.items():
#     print key, '||', value
word_in_name = pd.Series(word_in_name)
word_in_name
word_in_name[word_in_name.index.isin(['sports'])]
name2category=eval(''.join(open('../data/osm/name2category.txt').readlines()))

In [17]:
mapped_by_word = []
for tags in un_mapped:
    mapped = set()
    if 'name' in tags:
        for name in tags['name']:
            for word in name.split(' '):
                c = name2category.get(word.lower(),'')
                if c:
                    mapped.add(c)
    mapped_by_word.append(mapped)

In [18]:
new_mapped_by_word = [s for s in mapped_by_word if s]
len(new_mapped_by_word)

0

In [19]:
tags_in_subtree[7556]

{u'addr:city': [u'Washington'],
 u'addr:country': [u'US'],
 u'addr:postcode': [u'20011'],
 u'addr:state': [u'DC'],
 u'addr:street': [u'Georgia Avenue Northwest'],
 u'amenity': [u'fuel'],
 u'building': [u'roof'],
 u'name': [u'Valero']}

In [20]:
len_mapped = pd.Series(mapped_subtrees).apply(len)
[tags_in_subtree[i] for i in len_mapped[len_mapped>1].index.tolist()]
print pd.Series(new_mapped).apply(str).value_counts().sum()
pd.Series(new_mapped).apply(str).value_counts()

14782


set(['cycling facilities'])                                                 5132
set(['transportation'])                                                     4578
set(['outdoors and recreation'])                                            3335
set(['professional service'])                                                479
set(['residence'])                                                           369
set(['schools&university'])                                                  201
set(['retail shop'])                                                         195
set(['food'])                                                                119
set(['transportation', 'outdoors and recreation'])                           115
set(['professional service', 'cycling facilities'])                          114
set(['cycling facilities', 'outdoors and recreation'])                        58
set(['art'])                                                                  26
set(['nightlife spot'])     

In [21]:
def get_sum_categories(list_mapped):
    sum_categories = defaultdict(int)
    for mapped in list_mapped:
        for c in list(mapped):
            sum_categories[c]+=1
    return sum_categories
way_sum = get_sum_categories(way_mapped)
node_sum = get_sum_categories(node_mapped)
rltn_sum = get_sum_categories(rltn_mapped)
before_merge = pd.DataFrame((Counter(way_sum)+Counter(node_sum)+Counter(rltn_sum)).items(),columns=['category','osm_before_merge'])
after_merge = pd.DataFrame(get_sum_categories(mapped_subtrees).items(),columns=['category','osm_after_merge'])
before_merge.merge(after_merge).to_csv('../data/category_osm.csv')

In [98]:
with open('../data/osm/untransformed_tag_value.tsv', 'w') as f:
    for tag_value, cnt in untransformed_tag_value.items():
        f.write('{}\t{}\t{}\n'.format(tag_value.split('=')[0],tag_value.split('=')[1], cnt))